In [20]:
import pickle
import matplotlib.pyplot as plt
import numpy as np
import torch


In [ ]:
ls

cvar_planning.ipynb  model_improvement.ipynb  rl.ipynb
downsample.ipynb     model_vis.ipynb          uq_10min1hr_50samples.npy
explore.ipynb        planning.ipynb           workspace.ipynb


## Down sampling

In [4]:
# 10s per sample -> 5 minutes per sample

downsampled = []


for i in range(5):
    with open('/abiomed-tmp/intermediate/at_data_amicgs_{}.pkl'.format(i), 'rb') as f:
        data_amicgs = pickle.load(f)
    
    for j in range(len(data_amicgs)):
        struct = data_amicgs[j]
        id = struct['patient_id']
        data = struct['data']
        # Downsample by taking the average of every 30 samples
        if len(data) >= 30:
            # Reshape to group every 30 samples and take mean
            num_complete_groups = len(data) // 30
            reshaped_data = data[:num_complete_groups * 30].reshape(num_complete_groups, 30, -1)
            downsampled_data = reshaped_data.mean(axis=1)
            downsampled.append({
                'patient_id': id,
                'data': downsampled_data
            })
    
# # Save downsampled data as pickle
# with open('/abiomed-tmp/intermediate/at_5min_data.pkl', 'wb') as f:
#     pickle.dump(downsampled, f)

# print(f"Saved {len(downsampled)} downsampled patient records")



In [24]:
# 10s -> 10 minutes per sample

downsampled = []


for i in range(5):
    with open('/abiomed-tmp/intermediate/at_data_amicgs_{}.pkl'.format(i), 'rb') as f:
        data_amicgs = pickle.load(f)
    
    for j in range(len(data_amicgs)):
        struct = data_amicgs[j]
        id = struct['patient_id']
        data = struct['data']
        # Downsample by taking the average of every 60 samples
        if len(data) >= 60:
            # Reshape to group every 30 samples and take mean
            num_complete_groups = len(data) // 60
            reshaped_data = data[:num_complete_groups * 60].reshape(num_complete_groups, 60, -1)
            downsampled_data = reshaped_data.mean(axis=1)
            downsampled.append({
                'patient_id': id,
                'data': downsampled_data
            })
    
# # Save downsampled data as pickle
# with open('/abiomed-tmp/intermediate/at_10min_data.pkl', 'wb') as f:
#     pickle.dump(downsampled, f)

# print(f"Saved {len(downsampled)} downsampled patient records")

Saved 1325 downsampled patient records


In [5]:
downsampled = []


for i in range(5):
    with open('/abiomed/intermediate/low_score_amicgs/at_data_amicgs_{}.pkl'.format(i), 'rb') as f:
        data_amicgs = pickle.load(f)
    
    for j in range(len(data_amicgs)):
        struct = data_amicgs[j]
        id = struct['patient_id']
        data = struct['data']
        # Downsample by taking the average of every 60 samples
        if len(data) >= 60:
            # Reshape to group every 30 samples and take mean
            num_complete_groups = len(data) // 60
            reshaped_data = data[:num_complete_groups * 60].reshape(num_complete_groups, 60, -1)
            downsampled_data = reshaped_data.mean(axis=1)
            downsampled.append({
                'patient_id': id,
                'data': downsampled_data
            })

In [6]:
    
# Save downsampled data as pickle
with open('/abiomed/intermediate/low_score_amicgs/at_10min_data.pkl', 'wb') as f:
    pickle.dump(downsampled, f)

print(f"Saved {len(downsampled)} downsampled patient records")

Saved 436 downsampled patient records


# Windowing

In [9]:


def create_windows(data, window_size, stride):
    Xs = []

    if len(data) > window_size:  
        # Only create windows if we have enough data points
        i = 0
        while i < len(data) - window_size:
            Xs.append(data[i:i+window_size])
            i += stride
        Xs = torch.stack(Xs)
    return Xs

In [ ]:


with open('/abiomed/intermediate/at_10min_data.pkl', 'rb') as f:
    downsampled = pickle.load(f)

print("Normal data 10min", len(downsampled), downsampled[0]['data'].shape)
#80% for training, 20% for testing
train_downsampled = downsampled[:int(len(downsampled) * 0.8)]
test_downsampled = downsampled[int(len(downsampled) * 0.8):]



with open('/abiomed/intermediate/low_score_amicgs/at_10min_data.pkl', 'rb') as f:
    downsampled = pickle.load(f)

print("low score 10min", len(downsampled), downsampled[0]['data'].shape)
#80% for training, 20% for testing
train_downsampled = downsampled[:int(len(downsampled) * 0.8)]
test_downsampled = downsampled[int(len(downsampled) * 0.8):]


Normal data 10min 1325 torch.Size([4, 13])
low score 10min 436 torch.Size([50, 13])


In [7]:


with open('/abiomed/intermediate/at_10min_data.pkl', 'rb') as f:
    downsampled = pickle.load(f)

print("Normal data 10min", len(downsampled) , "samples", downsampled[0]['data'].shape)
#75% for training, 25% for testing + validation
train_downsampled_normal = downsampled[:int(len(downsampled) * 0.8)]
test_downsampled_normal = downsampled[int(len(downsampled) * 0.8):]


with open('/abiomed/intermediate/low_score_amicgs/at_10min_data.pkl', 'rb') as f:
    downsampled = pickle.load(f)

print("low score 10min", len(downsampled), "samples", downsampled[0]['data'].shape)
#75% for training, 25% for testing + validation
train_downsampled_low_score = downsampled[:int(len(downsampled) * 0.8)]
test_downsampled_low_score = downsampled[int(len(downsampled) * 0.8):]


train_all = train_downsampled_normal + train_downsampled_low_score
test_all = test_downsampled_normal + test_downsampled_low_score




Normal data 10min 1325 samples torch.Size([4, 13])
low score 10min 436 samples torch.Size([50, 13])


In [24]:

train_1hr = []
test_1hr = []

window_size = 12
stride = 3

for struct in train_all:  
    data = struct['data']
    xs = create_windows(data,window_size,stride)
    if len(xs) > 4: # more than 4 hours in length
        train_1hr.append(xs)

for struct in test_all:
    data = struct['data']
    xs = create_windows(data,window_size,stride)
    if len(xs) > 4: # more than 4 hours in length
        test_1hr.append(xs)

train_1hr = torch.cat(train_1hr)
test_1hr = torch.cat(test_1hr)

print("train_1hr.shape",train_1hr.shape)
print("test_1hr.shape",test_1hr.shape)

mean = torch.mean(train_1hr.reshape(-1, 13), axis=0)
std = torch.std(train_1hr.reshape(-1, 13), axis=0)

# randomly sample half of the test data for validation
val_data = test_1hr[torch.randperm(test_1hr.shape[0])[:len(test_1hr)//2]]

#dict_keys(['train', 'val', 'test', 'mean', 'std'])

with open('/abiomed/downsampled/10min_1hr_all_data.pkl', 'wb') as f:
    pickle.dump({'train': train_1hr, 'val': val_data, 'test': test_1hr, 'mean': mean, 'std': std}, f)

train_1hr.shape torch.Size([12051, 12, 13])
test_1hr.shape torch.Size([3876, 12, 13])


In [23]:
print(mean.shape)

torch.Size([13])


In [ ]:
# # input is 1hr, output is 1hr

# train_xs = []
# test_xs = []

# window_size = 24
# stride = 12

# for struct in train_downsampled:  
#     data = struct['data']
#     xs = create_windows(data,window_size,stride)
#     if len(xs) > 4: # more than 4 hours in length
#         train_xs.append(xs)

# for struct in test_downsampled:
#     data = struct['data']
#     xs = create_windows(data,window_size,stride)
#     if len(xs) > 4: # more than 4 hours in length
#         test_xs.append(xs)

# train_xs = torch.cat(train_xs)
# test_xs = torch.cat(test_xs)

# print("train_xs.shape",train_xs.shape)
# print("test_xs.shape",test_xs.shape)
# with open('/abiomed-tmp/downsampled/5min_1hr_window.pkl', 'wb') as f:
#     pickle.dump({'train': train_xs_1hr, 'test': test_xs_1hr}, f)

train_xs_1hr.shape torch.Size([4881, 24, 13])
test_xs_1hr.shape torch.Size([1440, 24, 13])


In [ ]:


with open('/abiomed-tmp/intermediate/at_10min_data.pkl', 'rb') as f:
    downsampled = pickle.load(f)

print(len(downsampled))
#80% for training, 20% for testing
train_downsampled = downsampled[:int(len(downsampled) * 0.8)]
test_downsampled = downsampled[int(len(downsampled) * 0.8):]

train_xs_1hr = []
test_xs_1hr = []

window_size = 60
stride = 30

for struct in train_downsampled:  
    data = struct['data']
    xs = create_windows(data,window_size,stride)
    if len(xs) > 4: # more than 4 hours in length
        train_xs_1hr.append(xs)

for struct in test_downsampled:
    data = struct['data']
    xs = create_windows(data,window_size,stride)
    if len(xs) > 4: # more than 4 hours in length
        test_xs_1hr.append(xs)

train_xs_1hr = torch.cat(train_xs_1hr)
test_xs_1hr = torch.cat(test_xs_1hr)

print("train_xs_1hr.shape",train_xs_1hr.shape)
print("test_xs_1hr.shape",test_xs_1hr.shape)


In [ ]:
# 1325 patients in total
# 896 greater than 1 hour
# 640 greater than 2 hour
# 357 greater than 5 hour
# 87 greater than 24 hour
